In [21]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
import src.dataset as ds
import pickle

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
WORDS_VECTORS_DIR = 'word_vectors/'
LYRICS_DIR = 'Data/'
GLOVE_DIR = os.path.join(WORDS_VECTORS_DIR, 'glove.6B')
TEXT_DATA = os.path.join(LYRICS_DIR, 'unified_lyrics_dump.txt')

MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

In [23]:
# load glove Word Embeddings
with open(os.path.join(WORDS_VECTORS_DIR, 'glove_embeddings.pickle'), 'rb') as f:
    embeddings_index = pickle.load(f)

In [16]:
with open(TEXT_DATA, 'r') as f:
    text = f.read()
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts([text])
sequences = tokenizer.texts_to_sequences([text])

In [30]:
# prepare embedding matrix
word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [4]:
# check what not exist in our corpus
def check_our_corpus(our_words):
    not_found = []
    for word in our_words:
        if word.encode() not in embeddings_index:    
            not_found.append(word)
    return not_found

In [17]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 7658 unique tokens.


In [18]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 7659
